In [ ]:
import os
import cv2
import time
import warnings
import random
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import segmentation_models_pytorch as smp
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
from torch.nn import functional as F
from torch.utils.data import (
    DataLoader, 
    Dataset
)
from matplotlib import pyplot as plt
from albumentations import (
    HorizontalFlip, 
    ShiftScaleRotate, 
    Normalize, 
    Resize, 
    Compose, 
    GaussNoise,
    RandomContrast,
    RandomGamma,
    RandomBrightness,
    OneOf,
    ElasticTransform,
    GridDistortion,
    OpticalDistortion
)
from albumentations.torch import ToTensor

In [ ]:
warnings.filterwarnings("ignore")
gpu = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
IMAGE_SIZE = 768
BATCH_SIZE = 8
GRAD_ACCUM = 4
MODEL_NAME = "se_resnext50_32x4d"

In [ ]:
def run_length_decode(rle, height=1024, width=1024, fill_value=1):
    component = np.zeros((height, width), np.float32)
    component = component.reshape(-1)
    
    rle = np.array([int(s) for s in rle.strip().split(' ')])
    rle = rle.reshape(-1, 2)
    
    start = 0
    for index, length in rle:
        start = start + index
        end = start + length
        component[start: end] = fill_value
        start = end
    
    component = component.reshape(width, height).T
    return component


def run_length_encode(component):
    component = component.T.flatten()
    
    start = np.where(component[1:] > component[:-1])[0] + 1
    end = np.where(component[:-1] > component[1:])[0] + 1
    length = end - start
    rle = []
    for i in range(len(length)):
        if i == 0:
            rle.extend([start[0], length[0]])
        else:
            rle.extend([start[i] - end[i-1], length[i]])
    rle = ' '.join([str(r) for r in rle])
    return rle

In [ ]:
class SIIMDataset(Dataset):
    def __init__(self, df, data_folder, size, mean, std, phase):
        self.df = df
        self.data_folder = data_folder
        self.size = size
        self.mean = mean
        self.std = std
        self.phase = phase
        self.transforms = get_transforms(phase, size, mean, std)
        self.gb = self.df.groupby('ImageId')
        self.fnames = list(self.gb.groups.keys())

    def __getitem__(self, idx):
        image_id = self.fnames[idx]
        df = self.gb.get_group(image_id)
        annotations = df[' EncodedPixels'].tolist()
        image_path = os.path.join(self.data_folder, image_id + ".png")
        image = cv2.imread(image_path)
        mask = np.zeros([1024, 1024])
        
        if annotations[0] != ' -1':
            for rle in annotations:
                mask += run_length_decode(rle)
                
        mask = (mask >= 1).astype('float32') 
        augmented = self.transforms(image=image, mask=mask)
        
        image = augmented['image']
        mask = augmented['mask']
        return image, mask

    def __len__(self):
        return len(self.fnames)


def get_transforms(phase, size, mean, std):
    list_transforms = []
    if phase == "train":
        list_transforms.extend([
            ShiftScaleRotate(
                shift_limit=0,  # no resizing
                scale_limit=0.1,
                rotate_limit=10, # rotate
                p=0.5,
                border_mode=cv2.BORDER_CONSTANT
            ),
            OneOf([
                RandomContrast(),
                RandomGamma(),
                RandomBrightness(),
                ], p=np.random.choice([.1, .2, .3])
            ),
            OneOf([
                ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
                GridDistortion(),
                OpticalDistortion(distort_limit=2, shift_limit=0.5),
                ], p=np.random.choice([.1, .2, .3])
            )
        ])
    list_transforms.extend([
        Resize(size, size),
        Normalize(mean=mean, std=std, p=1),
        ToTensor()
    ])

    list_trfms = Compose(list_transforms)
    return list_trfms



def provider(bag, fold, total_folds, data_folder, df_path, phase, size, mean=None, std=None, 
             batch_size=8, num_workers=4):
    df = pd.read_csv(df_path)
    
    df_with_mask = df[df[" EncodedPixels"] != " -1"]
    df_with_mask['has_mask'] = 1
    
    df_without_mask = df[df[" EncodedPixels"] == " -1"]
    df_without_mask['has_mask'] = 0
    df_without_mask_sampled = df_without_mask.sample(len(df_with_mask.drop_duplicates('ImageId')))
    
    df = pd.concat([df_with_mask, df_without_mask_sampled])
    
    kfold = StratifiedKFold(total_folds, shuffle=True, random_state=bag)
    train_idx, val_idx = list(kfold.split(df["ImageId"], df["has_mask"]))[fold]
    train_df, val_df = df.iloc[train_idx], df.iloc[val_idx]

    if phase == "train":
        df = train_df  
    else:
        df = val_df
    
    image_dataset = SIIMDataset(df, data_folder, size, mean, std, phase)
    dataloader = DataLoader(image_dataset, batch_size=batch_size, num_workers=num_workers, 
                            pin_memory=True, shuffle=True)
    return dataloader

In [ ]:
def dice_loss(input, target):
    input = torch.sigmoid(input)
    smooth = 1.0
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))


class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))
            
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()
        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        return loss.mean()


class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)

    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()

In [ ]:
def predict(X, threshold):
    X_p = np.copy(X)
    preds = (X_p > threshold).astype('uint8')
    return preds


def metric(probability, truth, threshold=0.5, reduction='none'):
    '''Calculates dice of positive and negative images seperately'''
    '''probability and truth must be torch tensors'''
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = (truth > 0.5).float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = torch.cat([dice_pos, dice_neg])

        dice_neg = np.nan_to_num(dice_neg.mean().item(), 0)
        dice_pos = np.nan_to_num(dice_pos.mean().item(), 0)
        dice = dice.mean().item()

        num_neg = len(neg_index)
        num_pos = len(pos_index)

    return dice, dice_neg, dice_pos, num_neg, num_pos


class Meter:
    '''A meter to keep track of iou and dice scores throughout an epoch'''
    def __init__(self, phase, epoch):
        self.base_threshold = 0.5
        self.base_dice_scores = []
        self.dice_neg_scores = []
        self.dice_pos_scores = []
        self.iou_scores = []

    def update(self, targets, outputs):
        probs = torch.sigmoid(outputs)
        dice, dice_neg, dice_pos, _, _ = metric(probs, targets, self.base_threshold)
        self.base_dice_scores.append(dice)
        self.dice_pos_scores.append(dice_pos)
        self.dice_neg_scores.append(dice_neg)
        preds = predict(probs, self.base_threshold)
        iou = compute_iou_batch(preds, targets, classes=[1])
        self.iou_scores.append(iou)

    def get_metrics(self):
        dice = np.mean(self.base_dice_scores)
        dice_neg = np.mean(self.dice_neg_scores)
        dice_pos = np.mean(self.dice_pos_scores)
        dices = [dice, dice_neg, dice_pos]
        iou = np.nanmean(self.iou_scores)
        return dices, iou

    
def epoch_log(phase, epoch, epoch_loss, meter, start):
    '''logging the metrics at the end of an epoch'''
    dices, iou = meter.get_metrics()
    dice, dice_neg, dice_pos = dices
    print("Loss: %0.4f | dice: %0.4f | dice_neg: %0.4f | dice_pos: %0.4f | IoU: %0.4f" % 
          (epoch_loss, dice, dice_neg, dice_pos, iou))
    return dice, iou


def compute_ious(pred, label, classes, ignore_index=255, only_present=True):
    '''computes iou for one ground truth mask and predicted mask'''
    pred[label == ignore_index] = 0
    ious = []
    for c in classes:
        label_c = label == c
        if only_present and np.sum(label_c) == 0:
            ious.append(np.nan)
            continue
        pred_c = pred == c
        intersection = np.logical_and(pred_c, label_c).sum()
        union = np.logical_or(pred_c, label_c).sum()
        if union != 0:
            ious.append(intersection / union)
    return ious if ious else [1]


def compute_iou_batch(outputs, labels, classes=None):
    '''computes mean iou for a batch of ground truth masks and predicted masks'''
    ious = []
    preds = np.copy(outputs) # copy is imp
    labels = np.array(labels) # tensor to np
    for pred, label in zip(preds, labels):
        ious.append(np.nanmean(compute_ious(pred, label, classes)))
    iou = np.nanmean(ious)
    return iou

In [ ]:
class Trainer(object):
    '''This class takes care of training and validation of our model'''
    def __init__(self, model, bag=0, fold=0, batch_size=BATCH_SIZE, grad_accum=GRAD_ACCUM, image_size=IMAGE_SIZE, 
                 max_epochs=50, lr=5e-4, early_stopping=7):
        self.bag = bag
        self.fold = fold
        self.total_folds = 5
        self.phases = ["train", "val"]
        
        self.device = 'cuda'
        self.num_workers = 8
        torch.set_default_tensor_type("torch.cuda.FloatTensor")
        cudnn.benchmark = True
        
        self.batch_size = {"train": batch_size, "val": batch_size}
        self.accumulation_steps = grad_accum
        self.num_epochs = max_epochs
        
        self.net = model.to(self.device)
        self.lr = lr
        self.criterion = MixedLoss(10.0, 2.0)
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode="max", patience=3, verbose=True)
        self.best_dice = float("-inf")

        self.dataloaders = {
            phase: provider(
                bag=bag,
                fold=fold,
                total_folds=5,
                data_folder=data_folder,
                df_path=train_rle_path,
                phase=phase,
                size=image_size,
                mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
                batch_size=self.batch_size[phase],
                num_workers=self.num_workers,
            )
            for phase in self.phases
        }
        self.losses = {phase: [] for phase in self.phases}
        self.iou_scores = {phase: [] for phase in self.phases}
        self.dice_scores = {phase: [] for phase in self.phases}
        self.early_stopping = early_stopping
        
    def forward(self, images, targets):
        images = images.to(self.device)
        masks = targets.to(self.device)
        outputs = self.net(images)
        loss = self.criterion(outputs, masks)
        return loss, outputs

    def iterate(self, epoch, phase):
        meter = Meter(phase, epoch)
        
        self.net.train(phase == "train")
        
        batch_size = self.batch_size[phase]
        dataloader = self.dataloaders[phase]
        total_batches = len(dataloader)

        running_loss = 0.0
        self.optimizer.zero_grad()

        start = time.strftime("%H:%M:%S")
        print(f"Starting epoch: {epoch} | phase: {phase} | ⏰: {start}")
        for itr, batch in tqdm(enumerate(dataloader), desc=phase, total=total_batches, leave=False):
            images, targets = batch

            if phase == "train":
                loss, outputs = self.forward(images, targets)
                loss.backward()
                if (itr + 1 ) % self.accumulation_steps == 0:
                    self.optimizer.step()
                    self.optimizer.zero_grad()
            else:
                with torch.no_grad():
                    loss, outputs = self.forward(images, targets)
            
            loss = loss / self.accumulation_steps
            running_loss += loss.item()
            outputs = outputs.detach().cpu()
            meter.update(targets, outputs)

        epoch_loss = (running_loss * self.accumulation_steps) / total_batches
        dice, iou = epoch_log(phase, epoch, epoch_loss, meter, start)
        self.losses[phase].append(epoch_loss)
        self.dice_scores[phase].append(dice)
        self.iou_scores[phase].append(iou)

        torch.cuda.empty_cache()
        return epoch_loss, dice

    def start(self):
        best_epoch = 0
        for epoch in range(self.num_epochs):
            self.iterate(epoch, "train")
            state = {
                "epoch": epoch,
                "best_dice": self.best_dice,
                "state_dict": self.net.state_dict(),
                "optimizer": self.optimizer.state_dict(),
            }
            val_loss, val_dice = self.iterate(epoch, "val")
            self.scheduler.step(val_dice)

            if val_dice > self.best_dice:
                print("******** New optimal found, saving state ********")
                state["best_dice"] = self.best_dice = val_dice
                torch.save(state, "./model_{}_size_{}_bag_{}_fold_{}.pth".format(self.net.name, IMAGE_SIZE, self.bag, self.fold))
                best_epoch = epoch
            else:
                if epoch - best_epoch > self.early_stopping:
                    print('********early stopping triggered')
                    break   

In [ ]:
class TestDataset(Dataset):
    def __init__(self, data_folder, df, size, mean, std, tta=4):
        self.data_folder = data_folder
        self.size = size
        self.fnames = list(df["ImageId"])
        self.num_samples = len(self.fnames)
        self.transform = Compose(
            [
                Resize(size, size),
                Normalize(mean=mean, std=std, p=1),
                ToTensor(),
            ]
        )

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.data_folder, fname + ".png")
        image = cv2.imread(path)
        images = self.transform(image=image)["image"]
        return images

    def __len__(self):
        return self.num_samples


def post_process(probability, threshold, min_size):
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((1024, 1024), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num


def plot(scores, name):
    plt.figure(figsize=(15, 5))
    plt.plot(range(len(scores["train"])), scores["train"], label=f'train {name}')
    plt.plot(range(len(scores["train"])), scores["val"], label=f'val {name}')
    plt.title(f'{name} plot'); plt.xlabel('Epoch'); plt.ylabel(f'{name}');
    plt.legend(); 
    plt.show()

In [ ]:
sample_submission_path = '../data/sample_submission.csv'
train_rle_path = '../data/train-rle.csv'
data_folder = "../data/train_png"
test_data_folder = "../data/test_png"

In [ ]:
test_preds = []

for bag in range(5):
    for fold in range(5):
        print("---- Start training on fold {}".format(fold))

        model = smp.Unet(MODEL_NAME, encoder_weights="imagenet", activation=None)
        model_trainer = Trainer(model, fold=fold)
        model_trainer.start()

        losses = model_trainer.losses
        dice_scores = model_trainer.dice_scores 
        iou_scores = model_trainer.iou_scores

        plot(losses, "BCE loss")
        plot(dice_scores, "Dice score")
        plot(iou_scores, "IoU score")
        plt.show()

        size = IMAGE_SIZE
        batch_size = BATCH_SIZE
        mean = (0.485, 0.456, 0.406)
        std = (0.229, 0.224, 0.225)
        num_workers = 8
        best_threshold = 0.5
        min_size = 3500
        device = 'cuda'

        df = pd.read_csv(sample_submission_path)
        testset = DataLoader(
            TestDataset(test_data_folder, df, size, mean, std),
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True,
        )

        model = model_trainer.net
        state = torch.load('./model_{}_size_{}_bag_{}_fold_{}.pth'.format(model.name, IMAGE_SIZE, bag, fold), 
                           map_location=lambda storage, loc: storage)
        model.load_state_dict(state["state_dict"])
        model.eval()
        fold_preds = []
        encoded_pixels = []
        for i, batch in enumerate(tqdm(testset)):
            with torch.no_grad():
                preds = torch.sigmoid(model(batch.to(device)))
            preds = preds.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
            fold_preds.append(preds)

            for probability in preds:
                if probability.shape != (1024, 1024):
                    probability = cv2.resize(probability, dsize=(1024, 1024), interpolation=cv2.INTER_LINEAR)

                prediction, num_predict = post_process(probability, best_threshold, min_size)

                if num_predict == 0:
                    encoded_pixels.append('-1')
                else:
                    r = run_length_encode(prediction)
                    encoded_pixels.append(r)

        fold_preds = np.vstack(fold_preds)
        df['EncodedPixels'] = encoded_pixels
        df.to_csv('model_{}_size_{}_bag_{}_fold_{}.csv'.format(model.name, IMAGE_SIZE, bag, fold), 
                  columns=['ImageId', 'EncodedPixels'], index=False)    

        test_preds.append(fold_preds)

In [ ]:
avg_preds = test_preds[0] / 5
for test_pred in test_preds[1:5]:
    avg_preds += test_pred / 5

encoded_pixels = []
for probability in avg_preds:
    if probability.shape != (1024, 1024):
        probability = cv2.resize(probability, dsize=(1024, 1024), interpolation=cv2.INTER_LINEAR)

    prediction, num_predict = post_process(probability, best_threshold, min_size)

    if num_predict == 0:
        encoded_pixels.append('-1')
    else:
        r = run_length_encode(prediction)
        encoded_pixels.append(r)

df['EncodedPixels'] = encoded_pixels
df.to_csv('model_{}_size_{}_avg.csv'.format(model.name, IMAGE_SIZE), 
          columns=['ImageId', 'EncodedPixels'], index=False)    

In [ ]:
avg_preds = test_preds[0] / len(test_preds)
for test_pred in test_preds[1:]:
    avg_preds += test_pred / len(test_preds)

encoded_pixels = []
for probability in avg_preds:
    if probability.shape != (1024, 1024):
        probability = cv2.resize(probability, dsize=(1024, 1024), interpolation=cv2.INTER_LINEAR)

    prediction, num_predict = post_process(probability, best_threshold, min_size)

    if num_predict == 0:
        encoded_pixels.append('-1')
    else:
        r = run_length_encode(prediction)
        encoded_pixels.append(r)

df['EncodedPixels'] = encoded_pixels
df.to_csv('model_{}_size_{}_bagged_avg.csv'.format(model.name, IMAGE_SIZE), 
          columns=['ImageId', 'EncodedPixels'], index=False)    